<a href="https://colab.research.google.com/github/dnrghks5583/Way-that-set-threshold-for-setting-PMD/blob/main/CALTECH-101/ResNet56_CALTECH_101.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

image_size = (164, 164)
input_shape = (164, 164, 3)
path = '/content/gdrive/MyDrive/Colab Notebooks/ResNet_졸과/GP/pickle_101_164'

Mounted at /content/gdrive


In [2]:
import os
import pathlib
import numpy as np
from tensorflow.keras import *
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Dropout, Flatten, BatchNormalization, Input, Add, ReLU, AveragePooling2D, GlobalAveragePooling2D, ZeroPadding2D
from tensorflow.keras.applications.resnet50 import ResNet50
from keras.preprocessing import image

image_size = (164, 164)
input_shape = (164, 164, 3)
num_classes = 102

In [3]:
import gzip
import pickle

os.chdir(path)

with gzip.open('caltech101_x_trainData.pickle', 'rb') as f:
    x_train = pickle.load(f)
    
with gzip.open('caltech101_x_testData.pickle', 'rb') as f:
    x_test = pickle.load(f)
    
with gzip.open('caltech101_y_trainData.pickle', 'rb') as f:
    y_train = pickle.load(f)
    
with gzip.open('caltech101_y_testData.pickle', 'rb') as f:
    y_test = pickle.load(f)

In [4]:

base_model = ResNet50(include_top = False, weights = 'imagenet')

x = base_model.output

shortcut = x
shortcut = Conv2D(filters = 2048, kernel_size = 1, strides = 1)(shortcut)
shortcut = BatchNormalization(axis = 3, epsilon=1.001e-5)(shortcut)

x = Conv2D(filters = 512, kernel_size = 1, strides = 1)(x)
x = BatchNormalization(axis = 3, epsilon=1.001e-5)(x)
x = Activation('relu')(x)

x = Conv2D(filters = 1024, kernel_size = 3, padding='SAME')(x)
x = BatchNormalization(axis = 3, epsilon=1.001e-5,)(x)
x = Activation('relu')(x)

x = Conv2D(filters = 2048, kernel_size = 1)(x)
x = BatchNormalization(axis = 3, epsilon=1.001e-5)(x)

x = Add()([shortcut, x])
x = Activation('relu')(x)

shortcut = x
shortcut = Conv2D(filters = 2048, kernel_size = 1, strides = 1)(shortcut)
shortcut = BatchNormalization(axis = 3, epsilon=1.001e-5)(shortcut)

x = Conv2D(filters = 512, kernel_size = 1, strides = 1)(x)
x = BatchNormalization(axis = 3, epsilon=1.001e-5)(x)
x = Activation('relu')(x)

x = Conv2D(filters = 1024, kernel_size = 3, padding='SAME')(x)
x = BatchNormalization(axis = 3, epsilon=1.001e-5,)(x)
x = Activation('relu')(x)

x = Conv2D(filters = 2048, kernel_size = 1)(x)
x = BatchNormalization(axis = 3, epsilon=1.001e-5)(x)

x = Add()([shortcut, x])
x = Activation('relu')(x)
x = GlobalAveragePooling2D()(x)

prediction = Dense(num_classes, activation='softmax')(x)
model = Model(inputs = base_model.input, outputs = prediction)
model.summary()

94773248/94765736 [==============================] - 1s 0us/step
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, None, None, 6 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, None, None, 6 256         conv1_conv[0][0]                 
_____________________________

In [5]:
def lr_schedule(epoch):
    """Learning Rate Schedule
    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.
    # Arguments
        epoch (int): The number of epochs
    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr



In [6]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.callbacks import ReduceLROnPlateau

lr_scheduler = LearningRateScheduler(lr_schedule)

lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)

callbacks = [lr_reducer, lr_scheduler]

model.compile(
    optimizer=Adam(learning_rate=lr_schedule(0)),
    loss = 'categorical_crossentropy',
    metrics = ['accuracy'])

Learning rate:  0.001


In [7]:
model.fit(x_train, y_train, validation_split = 0.1, batch_size = 32, epochs = 100, callbacks=callbacks)

Epoch 1/100
Learning rate:  0.001
206/206 [==============================] - 95s 279ms/step - loss: 2.9614 - accuracy: 0.3799 - val_loss: 5.1254 - val_accuracy: 0.1749
Epoch 2/100
Learning rate:  0.001
206/206 [==============================] - 55s 269ms/step - loss: 1.7224 - accuracy: 0.5639 - val_loss: 2.2151 - val_accuracy: 0.5137
Epoch 3/100
Learning rate:  0.001
206/206 [==============================] - 57s 275ms/step - loss: 1.0260 - accuracy: 0.7198 - val_loss: 1.6496 - val_accuracy: 0.6120
Epoch 4/100
Learning rate:  0.001
206/206 [==============================] - 57s 278ms/step - loss: 0.6305 - accuracy: 0.8193 - val_loss: 1.2634 - val_accuracy: 0.7063
Epoch 5/100
Learning rate:  0.001
206/206 [==============================] - 58s 279ms/step - loss: 0.4143 - accuracy: 0.8782 - val_loss: 1.8483 - val_accuracy: 0.5943
Epoch 6/100
Learning rate:  0.001
206/206 [==============================] - 58s 280ms/step - loss: 0.2954 - accuracy: 0.9128 - val_loss: 1.7438 - val_accuracy:

In [8]:
model.evaluate(x_test, y_test)

58/58 [==============================] - 5s 94ms/step - loss: 0.8956 - accuracy: 0.8403


[0.8956156373023987, 0.8403499126434326]